In [2]:
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Trade_Functions import *
import numpy as np

In [3]:
cal = get_calendar()

In [8]:
def pick_trade(RunDate,NumStocks,Money,cal=get_calendar(),holdTime=0,TimeInput=5,NDXfile="Nasdaq100MembersJan2024.csv",debug=False):
    
    print(RunDate)
    
    LongStockList = list()
    ShortStockList = list()
    
    PctChange = get_SortedPctChange(RunDate,TimeInput,NDXfile,debug=debug)
    
    
    LongStockList = PctChange["Ticker"].iloc[:NumStocks]
    LongReturns = getLongReturns(RunDate,LongStockList,holdTime,debug=debug)
    TotalReturn= (1+LongReturns[0])*(Money*0.75) + (1+LongReturns[1])*(Money*0.25)
    
    
    d = {   "Stock":[LongStockList[0],LongStockList[1]], 
     "PctChange":[PctChange["PctChange"][0],PctChange["PctChange"][1]],  
     "Bought_At":[f"{RunDate}_Open",f"{RunDate}_Open"], 
     "BuyPrice":[getPrice(RunDate,LongStockList[0],"Open"),getPrice(RunDate,LongStockList[1],"Open")],
    "Sold_At":[f"{getDateOffset(RunDate,1)}_Close",f"{getDateOffset(RunDate,1)}_Close"],
    "SellPrice":[getPrice(getDateOffset(RunDate,1),LongStockList[0],"Close"),getPrice(getDateOffset(RunDate,1),LongStockList[1],"Close")]}
    
    #ShortStockList =  PctChange["Ticker"].iloc[len(PctChange["Ticker"])-NumStocks:]
    #ShortReturns = getShortReturns(RunDate,ShortStockList,holdTime,debug=debug)
    #TotalReturn += (1+ShortReturns[0])*(Money*0.125) + (1+ShortReturns[1])*(Money*0.375)

    print("Total Return: ", TotalReturn)
    
    return TotalReturn, pd.DataFrame(d)

In [9]:
pick_trade('2024-10-03',2,1)

2024-10-03
PDD
147.0
154.27000427246094
Total Return:  1.037512186581349


(1.037512186581349,
   Stock  PctChange        Bought_At    BuyPrice           Sold_At   SellPrice
 0   PDD   0.180139  2024-10-03_Open  147.000000  2024-10-04_Close  154.270004
 1  FANG   0.073611  2024-10-03_Open  181.869995  2024-10-04_Close  194.610001)

In [13]:
TimeInput = 4
NDXfile="Nasdaq100MembersJan2024.csv"
NumStocks=2
holdTime=1

In [20]:
Money = 1
TotalReturnList = list()
DateReturnList = list()

DateRange=['2024-01-02','2024-04-01']
DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]

TradeHistory=pd.DataFrame([])
for day in cal[DateRange[0]:DateRange[1]]:
    Money,trades = pick_trade(RunDate=day,NumStocks=NumStocks,Money=Money,cal=cal,TimeInput=TimeInput,NDXfile=NDXfile,holdTime=holdTime,debug=False)
    TradeHistory=pd.concat([TradeHistory,trades],axis=0)
    TotalReturnList.append(Money)
    DateReturnList.append(day)

dfOut = pd.DataFrame({'Date':DateReturnList,'Return':TotalReturnList})







2024-01-02
REGN
873.9099731445312
917.5599975585938
Total Return:  1.0219356207936805
2024-01-03
MRNA
112.51000213623047
108.66000366210938
Total Return:  0.9926019921466892
2024-01-04
MRNA
109.01000213623047
111.12000274658203
Total Return:  1.0082461954079966
2024-01-05
MRNA
107.7699966430664
115.44000244140625
Total Return:  1.069303844813997
2024-01-08
MRVL
60.0
63.470001220703125
Total Return:  1.1599597361242142
2024-01-09
SMCI
31.83300018310547
34.237998962402344
Total Return:  1.2271567773974361
2024-01-10
SMCI
34.70000076293945
34.33100128173828
Total Return:  1.2281401938561205
2024-01-11
SMCI
34.48400115966797
33.95600128173828
Total Return:  1.2133340288660905
2024-01-12
PANW
324.510009765625
332.04998779296875
Total Return:  1.2322589406562012
2024-01-16
ISRG
363.3900146484375
358.8800048828125
Total Return:  1.2224403556377421
2024-01-17
AMD
158.4499969482422
162.6699981689453
Total Return:  1.254378776080971
2024-01-18
AMD
166.1999969482422
174.22999572753906
Total Retur

In [21]:
TradeHistory

Stock  PctChange        Bought_At    BuyPrice           Sold_At   SellPrice
0   REGN   0.033854  2024-01-02_Open  873.909973  2024-01-03_Close  917.559998
1    AMD   0.027892  2024-01-02_Open  144.279999  2024-01-03_Close  135.320007
0   MRNA   0.116847  2024-01-03_Open  112.510002  2024-01-04_Close  108.660004
1    KHC   0.043812  2024-01-03_Open   38.244999  2024-01-04_Close   37.779999
0   MRNA   0.099099  2024-01-04_Open  109.010002  2024-01-05_Close  111.120003
..   ...        ...              ...         ...               ...         ...
1   SMCI   0.163218  2024-03-26_Open  108.699997  2024-03-27_Close  102.329002
0     MU   0.081475  2024-03-27_Open  119.300003  2024-03-28_Close  117.889999
1    CEG   0.064779  2024-03-27_Open  188.080002  2024-03-28_Close  184.850006
0   MRVL   0.086715  2024-03-28_Open   72.000000  2024-04-01_Close   73.199997
1     MU   0.082025  2024-03-28_Open  119.440002  2024-04-01_Close  124.300003

[122 rows x 6 columns]

In [ ]:
PrctReturnList=list()
retList= [1]+TotalReturnList
for i in range(1,len(retList)-1):
    PrctReturnList.append((TotalReturnList[i]-TotalReturnList[i-1])/TotalReturnList[i-1])
    
PrctReturnList = np.array(PrctReturnList)

In [ ]:
dfOut

In [ ]:
PrctReturnList

In [ ]:
print(np.mean(PrctReturnList[PrctReturnList <0]))
print(len(PrctReturnList[PrctReturnList <0]))

In [ ]:
print(np.mean(PrctReturnList[PrctReturnList >0]))
print(len(PrctReturnList[PrctReturnList >0]))

In [ ]:
dfOut.to_csv("ReturnData.csv")


In [22]:
TradeHistory.to_csv("TradeHistory.csv")